<a href="https://colab.research.google.com/github/HonkinWaffles/Google-Colab-Notebooks/blob/main/Honkin_Real_ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESRGRAN
Real-ESRGAN - https://github.com/xinntao/Real-ESRGAN

Model Data - v4

## Initial Configuration


### Setup

In [ ]:
!pip install wget

import os, torch, gc, shutil, wget
from getpass import getpass
from pathlib import Path

#Login to GDRIVE for file access
LOGIN_GDRIVE = True
if LOGIN_GDRIVE and not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

## Make directory structure in drive to set everything up correctly
!mkdir -p /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/OUTPUT 

# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Download the pre-trained model
path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/RealESRGAN_x4.pth'
path = Path(path_to_file)

if path.is_file():
    shutil.copy2('/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/RealESRGAN_x4plus.pth', 'experiments/pretrained_models/RealESRGAN_x4plus.pth')
    print('Copying local version of models')
else:
    wget.download('https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth', 'experiments/pretrained_models/RealESRGAN_x4plus.pth')
    print('Downloading Models from internet - Consider downloading and adding it to `/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP/`')

## Make directory structure in drive to set everything up correctly
!mkdir -p /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/SETUP /content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/OUTPUT 


## Basic Upscale

In [ ]:

#@markdown ---
#@markdown Size of final image 
OUT_SCALE = 4 #@param {type:'integer'}
#@markdown Precision during inference (16-32) 
FP = "32" #@param {type:'string'}
#@markdown ---


!python inference_realesrgan.py \
  -n RealESRGAN_x4plus \
  -i upload \
  --fp$FP \
  --outscale $OUT_SCALE \
  --face_enhance 
# Arguments
# -n, --model_name: Model names
# -i, --input: input folder or image
# --outscale: Output scale, can be arbitrary scale factore. 

#### Download

In [ ]:
# Download the results
zip_filename = 'Real-ESRGAN_result.zip'
if os.path.exists(zip_filename):
  os.remove(zip_filename)
os.system(f"zip -r -j {zip_filename} results/*")
files.download(zip_filename)

FileNotFoundError: ignored

## Bulk Upscale


### Upload

In [ ]:
import os
from google.colab import files
import shutil
import zipfile

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)

# Download the pre-trained model
FOLDER_NAME = "22-09-04-Personal" #@param {type:'string'}

path_to_file = '/content/drive/MyDrive/GOOGLE_COLAB/STABLE_DIFF/TXT2IMG/' + FOLDER_NAME + '/'
path = Path(path_to_file)
print(path_to_file)
print(path)

if path.is_dir():
    # Copy photos from drive
    isExist = os.path.exists(upload_folder)
    if isExist:
      shutil.rmtree(upload_folder)
      print("The upload directory has been removed!")
    shutil.copytree(path_to_file, 'upload')
    print('Copying images over')
else:
    #Upload photos via Colab
    isExist = os.path.exists(upload_folder)
    if not isExist:
      os.mkdir(upload_folder)
      print("The upload directory is created!")
    uploaded = files.upload()
    for filename in uploaded.keys():
      dst_path = os.path.join(upload_folder, filename)
      print(f'move {filename} to {dst_path}')
      shutil.move(filename, dst_path)

### Run ESRGRAN


In [ ]:
import glob
import os
#@markdown ---
#@markdown Size of final image 
OUT_SCALE = 4 #@param {type:'integer'}
#@markdown Precision during inference (16-32) 
FP = "32" #@param {type:'string'}
#@markdown ---


## Loop through each image inside the subfolders
rootdir = 'upload'
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        OUTPUT = d.rsplit('/', 1)[1]
        OUTPUT = (f"'results/" + OUTPUT +  "'")
        d = f'"{d}"'
        !python  inference_realesrgan.py \
        -n RealESRGAN_x4plus \
        -i $d \
        --fp$FP \
        --output $OUTPUT \
        --outscale $OUT_SCALE  \
        --face_enhance 


# Copy results back to drive
results_up = "/content/drive/MyDrive/GOOGLE_COLAB/ESRGAN/OUTPUT/" + FOLDER_NAME + "/"
isExist = os.path.exists(results_up)
if not isExist:
  os.mkdir(results_up)
  print("The upload directory is created!")

!cp -r /content/Real-ESRGAN/results/* $results_up
